# Roteiro de viagem com LangChain
___

Este notebook tem como objetivo entender passo a passo como construir um sistema com o LangChain.
Criar um assistente que, dado um interesse por um tipo de destino -- `praias` ou `ecoturismo`, ou outro qualquer,
crie um roteiro de viagem.

1. Cidade recomendada + motivo da recomendação
2. Restaurantes na cidade
3. Passeios culturais

___

### Parte 1: Setup e Imports

- Carregando as bibliotecas e OpenAI key

In [1]:
import os
import json

from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, PydanticOutputParser
from pydantic import BaseModel, Field

In [2]:
load_dotenv(find_dotenv())

openai_api_key = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(
                    model="gpt-4.1",
                    temperature=0,
                    api_key=openai_api_key
                )

### Parte 2: Testar apenas o modelo com um prompt simples
___

In [3]:
response = model.invoke("Qual a cidade mais visitada no Brasil para praticantes do ecoturismo?")
print(response.content)

A cidade mais visitada do Brasil por praticantes de **ecoturismo** é **Bonito**, localizada no estado de Mato Grosso do Sul. Bonito é reconhecida nacional e internacionalmente por suas águas cristalinas, grutas, cachoeiras, trilhas e uma grande variedade de atividades de turismo sustentável, como flutuação em rios, mergulho, rapel, observação de aves e exploração de cavernas.

Além de Bonito, outros destinos importantes para o ecoturismo no Brasil incluem:

- **Foz do Iguaçu (PR)** – famosa pelas Cataratas do Iguaçu e o Parque Nacional do Iguaçu.
- **Chapada Diamantina (BA)** – conhecida por suas trilhas, cachoeiras e grutas.
- **Chapada dos Veadeiros (GO)** – famosa por suas paisagens de cerrado, cachoeiras e trilhas.
- **Amazônia (AM/PA)** – com experiências de turismo de natureza e contato com a floresta.

No entanto, **Bonito** é amplamente considerada a principal referência nacional em ecoturismo, sendo frequentemente premiada e reconhecida por sua infraestrutura voltada para o tu

### Parte3: Criar Prompt Estruturado com Pydantic + JsonOutputParser
___

"* `{interesse}`: recomendação de cidade + motivo da recomendação"

In [4]:
class Destino(BaseModel):
    cidade: str = Field(description="Nome da cidade recomendada para o tipo de interesse do usuário.")
    motivo: str = Field(description="Motivo da recomendação da cidade para o interesse específicado pelo usuário.")

parser_destino = PydanticOutputParser(pydantic_object=Destino) # PydanticOutputParser é um parser que transforma a saída do modelo em um objeto Pydantic.

prompt_destino = ChatPromptTemplate.from_template(
    """Sugira uma cidade com base no interesse do usuário, {interesse}.
    Explique o motivo da sugestão da cidade.\n{format_instructions}""",
    partial_variables={"format_instructions": parser_destino.get_format_instructions()}
)

chain_destino = prompt_destino | model | parser_destino

response_destino = chain_destino.invoke({"interesse": "ecoturismo"}) # objeto pydantic

In [5]:
print(response_destino)
print(type(response_destino))

cidade='Bonito' motivo='Bonito, no Mato Grosso do Sul, é reconhecida internacionalmente como um dos melhores destinos de ecoturismo do Brasil. A cidade oferece uma grande variedade de atividades em meio à natureza, como mergulho em rios de águas cristalinas, trilhas, grutas, cachoeiras e observação de fauna e flora, tudo com forte compromisso com a sustentabilidade ambiental.'
<class '__main__.Destino'>


In [8]:
cidade_recomendada = response_destino.cidade
motivo_recomendacao = response_destino.motivo

print(f"Cidade recomendada: {cidade_recomendada}\n")
print(f"Motivo da recomendação: {motivo_recomendacao}")

Cidade recomendada: Bonito

Motivo da recomendação: Bonito, no Mato Grosso do Sul, é reconhecida internacionalmente como um dos melhores destinos de ecoturismo do Brasil. A cidade oferece uma grande variedade de atividades em meio à natureza, como mergulho em rios de águas cristalinas, trilhas, grutas, cachoeiras e observação de fauna e flora, tudo com forte compromisso com a sustentabilidade ambiental.


In [9]:
response_dict_json_schema = response_destino.model_json_schema()
response_json_schema = json.dumps(
    response_dict_json_schema, 
    indent=2,          # Adiciona indentação para legibilidade
    ensure_ascii=False # Garante que caracteres como "ç" e "ã" sejam exibidos corretamente
)
print(response_json_schema)

{
  "properties": {
    "cidade": {
      "description": "Nome da cidade recomendada para o tipo de interesse do usuário.",
      "title": "Cidade",
      "type": "string"
    },
    "motivo": {
      "description": "Motivo da recomendação da cidade para o interesse específicado pelo usuário.",
      "title": "Motivo",
      "type": "string"
    }
  },
  "required": [
    "cidade",
    "motivo"
  ],
  "title": "Destino",
  "type": "object"
}


#### Parte 3.1 - Saída e análise da primeira chain:

Sugestão de cidade por `{interesse}` e `motivo da recomendação`

In [10]:
# Supondo que seu código anterior já foi executado:
# model = ...
# parser_destino = ...
# prompt_destino = ...
# chain_destino = prompt_destino | model | parser_destino
# response = chain_destino.invoke({"interesse": "ecoturismo"})

# 1. Inspecionando o tipo e acessando os atributos
print(f"O tipo da variável 'response' é: {type(response_destino)}")
print("-" * 40)
print(f"Cidade Sugerida: {response_destino.cidade}")
print(f"Motivo da Sugestão: {response_destino.motivo}")
print("-" * 40)


# 2. Melhorando a visualização com .model_dump() e json.dumps()
print("\n✅ Saída formatada como JSON (ideal para logs e debug):")

# Converte o objeto Pydantic para um dicionário Python
response_dict = response_destino.model_dump()

# Usa json.dumps para formatar o dicionário como uma string JSON "bonita"
json_output = json.dumps(
    response_dict, 
    indent=2,          # Adiciona indentação para legibilidade
    ensure_ascii=False # Garante que caracteres como "ç" e "ã" sejam exibidos corretamente
)

print(json_output)


O tipo da variável 'response' é: <class '__main__.Destino'>
----------------------------------------
Cidade Sugerida: Bonito
Motivo da Sugestão: Bonito, no Mato Grosso do Sul, é reconhecida internacionalmente como um dos melhores destinos de ecoturismo do Brasil. A cidade oferece uma grande variedade de atividades em meio à natureza, como mergulho em rios de águas cristalinas, trilhas, grutas, cachoeiras e observação de fauna e flora, tudo com forte compromisso com a sustentabilidade ambiental.
----------------------------------------

✅ Saída formatada como JSON (ideal para logs e debug):
{
  "cidade": "Bonito",
  "motivo": "Bonito, no Mato Grosso do Sul, é reconhecida internacionalmente como um dos melhores destinos de ecoturismo do Brasil. A cidade oferece uma grande variedade de atividades em meio à natureza, como mergulho em rios de águas cristalinas, trilhas, grutas, cachoeiras e observação de fauna e flora, tudo com forte compromisso com a sustentabilidade ambiental."
}


### Parte 4 - Encadear a chain de restaurante
___ 

Dado a saída da chain anterior, que foi a cidade baseada no interesse do usuário, agora é escolher restaurantes de boa qualidade caseiros e um pouco mais sofisticados.

In [14]:
class Restaurante(BaseModel):
    nome: str = Field(description="Nome do restaurante.")
    tipo: str = Field(description="Tipo de culinária do restaurante.")
    nota: float = Field(description="Nota do restaurante.")
    preco: float = Field(description="Preço médio do prato para duas pessoas.")
    descricao: str = Field(description="Descrição do restaurante.")


class ListaRestaurantes(BaseModel):
    restaurantes: list[Restaurante] = Field(description="Lista de restaurantes sugeridos")

parser_restaurante = PydanticOutputParser(pydantic_object=ListaRestaurantes)

prompt_restaurante = ChatPromptTemplate.from_template(
    """Você é um assistente especialista em gastronomia e trabalha para a agência
    de viagens.

Para a cidade {cidade}, sugira 3 restaurantes de boa qualidade caseiros e 3 mais sofisticados.

Siga estritamente as instruções de formatação abaixo:
{format_instructions}
""",
)

chain_restaurante = prompt_restaurante | model | parser_restaurante
cidade = response_destino.cidade

response_restaurante = chain_restaurante.invoke({
    "cidade": cidade,
    "format_instructions": parser_restaurante.get_format_instructions()
})

#*A saída será um objeto ListaRestaurantes(Pydantic)
print(response_restaurante)

#* Podemos acessa a lista facilmente
for restaurante in response_restaurante.restaurantes:
    print(f"--- \nNome: {restaurante.nome}\nTipo: {restaurante.tipo}\nNota: {restaurante.nota}")

restaurantes=[Restaurante(nome='Casa do João', tipo='Culinária regional caseira', nota=4.7, preco=120.0, descricao='Restaurante tradicional de Bonito, famoso pelo ambiente acolhedor e pratos típicos sul-mato-grossenses preparados de forma caseira, como o pintado à urucum e a traíra sem espinha.'), Restaurante(nome='Restaurante da Zézinha', tipo='Culinária caseira brasileira', nota=4.6, preco=90.0, descricao='Ambiente simples e familiar, com buffet de comida caseira, incluindo opções de peixes regionais, carnes e saladas frescas. Muito procurado por moradores e turistas.'), Restaurante(nome='Cantinho do Peixe', tipo='Culinária regional caseira', nota=4.5, preco=100.0, descricao='Especializado em peixes de água doce preparados de maneira tradicional, oferece pratos fartos e saborosos em ambiente descontraído e familiar.'), Restaurante(nome='Juanita Restaurante', tipo='Culinária regional sofisticada', nota=4.8, preco=180.0, descricao='Restaurante sofisticado com ambiente elegante, especia

### Parte 5 - Encadear com passeios culturais - (texto livre)
___

In [15]:
prompt_passeios = ChatPromptTemplate.from_template(
    "Sugira 3 passeios culturais imperdíveis na cidade de {cidade}, com descrição de cada um."
)

chain_passeios = prompt_passeios | model | StrOutputParser()
cidade = response_destino.cidade

chain_passeios.invoke({"cidade": cidade})


'Claro! Embora Bonito, no Mato Grosso do Sul, seja mais famosa por suas belezas naturais e ecoturismo, a cidade também oferece experiências culturais interessantes. Aqui estão três passeios culturais imperdíveis em Bonito:\n\n---\n\n**1. Projeto Jiboia**  \nO Projeto Jiboia é uma iniciativa de educação ambiental que visa desmistificar o medo das serpentes e promover a conservação desses animais. Durante a visita, os participantes assistem a uma palestra divertida e informativa sobre a importância das cobras para o ecossistema, aprendem a diferenciar espécies peçonhentas e não peçonhentas e, ao final, têm a oportunidade de interagir e tirar fotos com uma jiboia. É uma experiência educativa, ideal para todas as idades, que une cultura, ciência e conservação ambiental.\n\n---\n\n**2. Casa da Memória Raída**  \nA Casa da Memória Raída é um espaço cultural dedicado à preservação da história e das tradições de Bonito e região. O local reúne objetos antigos, fotografias, documentos e relatos 

### Parte 6 - Montar a versão encadeada passo a passo:
___

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

chain_destino = prompt_destino | model | parser_destino

map_chain = RunnableParallel(
    destino=chain_destino,
    original_input=RunnablePassthrough()
)

full_chain = map_chain | RunnableParallel(
    destino=itemgetter("destino"),
    restaurantes=itemgetter("destino") | chain_restaurante,
    passeios=itemgetter("destino") | chain_passeios
)
